# CARGA INICIAL Y CONFIGURACION (importar archivo)

In [11]:
import pandas as pd
import numpy as np

try:
    df = pd.read_csv('impom24.csv', sep=';', decimal=',', encoding='latin-1')
    print("Carga de datos exitosa.")
    print("\nPrimeras 5 filas: ")
    print(df.head())
    print("n\Informacion general del DataFrame: ")
    df.info()
except FileNotFoundError:
    print("Error: Asegurate que la ruta del archivo CSV es correcta y que estas usando el archivo de datos.")

Carga de datos exitosa.

Primeras 5 filas: 
    Año  mes      NCM  Porg  Pnet(kg)  FOB(USD)  Flete(USD)  Seguro(USD)  \
0  2024    1  1012100   200     500.0   6000.00     3500.00        47.50   
1  2024    1  1012100   212    2500.0  19500.00    26000.01       455.01   
2  2024    1  1012100   417     300.0  33490.76     3256.80       367.48   
3  2024    1  1012900   200     450.0   5000.00     2500.00        75.00   
4  2024    1  1012900   423    1000.0   9800.00     6513.60       168.27   

   CIF(USD)  
0   9547.50  
1  45955.02  
2  37115.04  
3   7575.00  
4  16481.87  
n\Informacion general del DataFrame: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347874 entries, 0 to 347873
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Año          347874 non-null  int64  
 1   mes          347874 non-null  int64  
 2   NCM          347874 non-null  int64  
 3   Porg         347874 non-null  int64  
 4   Pnet

# INSPECCION Y ESTANDARIZACION DE COLUMNAS
## a-Renombrar columnas clave

In [12]:
# Mapeo de columnas según el documento que adjunto
nuevos_nombres = {
    'NCM': 'NCM_8_digitos',
    'Porg': 'Pais_Origen_Cod',
    'Pnet(kg)': 'Peso_Neto_kg',
    'FOB (USD)': 'Valor_FOB_USD',
    'Flete (USD)': 'Flete_USD',
    'Seguro (USD)': 'Seguro_USD',
    'CIF (USD)': 'Valor_CIF_USD' # Esta será la variable objetivo (Target)
}

df.rename(columns=nuevos_nombres, inplace=True)
print("Columnas renombradas:")
print(df.columns.tolist())

Columnas renombradas:
['Año', 'mes', 'NCM_8_digitos', 'Pais_Origen_Cod', 'Peso_Neto_kg', 'FOB(USD)', 'Flete(USD)', 'Seguro(USD)', 'CIF(USD)']


## b-Revisar tipos de datos

In [15]:
# Convertir NCM_8_digitos y País_Origen_Cod a enteros/cadenas para asegurar el formato
df['NCM_8_digitos'] = df['NCM_8_digitos'].astype(str).str.zfill(8) # Asegurar 8 dígitos con ceros a la izquierda
df['Pais_Origen_Cod'] = df['Pais_Origen_Cod'].astype(str).str.zfill(3) # Código del país
df['Año'] = df['Año'].astype(int)
df['mes'] = df['mes'].astype(int)

# Verificar si las columnas de valor son numéricas (deberían serlo si la carga fue correcta)
print("\nTipos de datos actualizados:")
print(df.dtypes)


Tipos de datos actualizados:
Año                  int64
mes                  int64
NCM_8_digitos       object
Pais_Origen_Cod     object
Peso_Neto_kg       float64
FOB(USD)           float64
Flete(USD)         float64
Seguro(USD)        float64
CIF(USD)           float64
dtype: object


# MANEJO DE DATOS FALTANTES (NaN) Y ATIPICOS
## a-Valores nulos (NaN)

### Es raro encontrar NaN en este dataset, pero si aparecen en las columnas de valor, lo más seguro es reemplazarlos por cero (0), asumiendo que un valor faltante significa que el costo no aplicó o fue cero en ese registro.

In [17]:
columnas_valor = ['Peso_Neto_kg', 'FOB(USD)', 'Flete(USD)', 'Seguro(USD)', 'CIF(USD)']

# Rellenar valores nulos con 0 en las columnas de valor
df[columnas_valor] = df[columnas_valor].fillna(0)

print(f"\nConteo de valores nulos después de la limpieza: \n{df.isnull().sum()}")


Conteo de valores nulos después de la limpieza: 
Año                0
mes                0
NCM_8_digitos      0
Pais_Origen_Cod    0
Peso_Neto_kg       0
FOB(USD)           0
Flete(USD)         0
Seguro(USD)        0
CIF(USD)           0
dtype: int64


## b-Registros irrelevantes o cero

### En comercio exterior, es común tener registros donde el Valor CIF es positivo, pero el Peso Neto es cero. Esto ocurre con productos digitales (software, licencias) o bienes de poco peso/alto valor.
### Para un análisis de valor, se pueden mantener. Para modelos que dependan de la densidad o el precio por kg, hay que considerar eliminarlos o manejarlos por separado.

In [20]:
# Filtrar registros donde todas las variables de valor sean cero, ya que no aportan a un modelo de predicción.
df_limpio = df[
    (df['CIF(USD)'] > 0) | 
    (df['FOB(USD)'] > 0)
].copy()

# Opcional: Aislar registros de Peso Neto 0
# Si queremos hacer un análisis posterior de alto valor/cero peso
df_alto_valor_cero_peso = df_limpio[df_limpio['Peso_Neto_kg'] == 0].copy()

print(f"\nTotal de registros originales: {len(df)}")
print(f"Total de registros limpios (CIF o FOB > 0): {len(df_limpio)}")


Total de registros originales: 347874
Total de registros limpios (CIF o FOB > 0): 347874


# CREACION DE VARIABLES
### Crearemos las variables auxiliares más útiles para cualquier modelo predictivo en comercio exterior: el precio unitario (por kg) y el costo logístico como porcentaje.

## a-Precio unitario (USD/KG)
### (Esta es una métrica clave para el ML, ya que le enseña al modelo el "valor intrínseco" del producto.)

In [23]:
# Evitar división por cero: si Peso_Neto_kg es 0, el Precio_Unitario será NaN.
# Si es 0, podemos asignar un valor muy alto para modelarlo como atípico, o simplemente dejarlo NaN.
# En este caso, lo dejamos NaN y lo gestionamos luego.

df_limpio['Precio_Unitario_USD_kg'] = df_limpio['FOB(USD)'] / df_limpio['Peso_Neto_kg']

# Rellenamos los NaN generados (donde Peso_Neto_kg era 0) con un valor que represente su valor único,
# como el valor FOB mismo (es decir, el costo por unidad es su costo total) o simplemente 0, 
# dependiendo de cómo se trate el subconjunto de "cero peso".

# Usaremos un gran número (como el máximo) o simplemente 0. Vamos con 0 por simplicidad, 
# asumiendo que los registros con peso 0 se analizan por separado o se excluyen de este cálculo.
df_limpio['Precio_Unitario_USD_kg'] = df_limpio['Precio_Unitario_USD_kg'].fillna(0)

## b-Creación de categorías NCM

### Agruparemos el NCM de 8 dígitos en categorías de 2 y 4 dígitos, lo cual es útil para análisis y modelos más amplios.

In [24]:
# NCM de 2 dígitos (Capítulo)
df_limpio['NCM_2_digitos'] = df_limpio['NCM_8_digitos'].str[:2]

# NCM de 4 dígitos (Partida)
df_limpio['NCM_4_digitos'] = df_limpio['NCM_8_digitos'].str[:4]

# RESULTADO FINAL

## El dataset df_limpio está ahora preparado: tienes la variable objetivo (CIF(USD)), features de valor (FOB(USD)), Flete(USD)), features categóricas (NCM_8_digitos, Pais_Origen_Cod) y features de tiempo (año, Mes).

In [25]:
print("\nDataFrame Limpio y Preparado (Primeras filas):")
print(df_limpio.head())
print(f"\nEl DataFrame final tiene {len(df_limpio)} registros y {len(df_limpio.columns)} columnas.")

# Guía para el próximo paso: guardar el dataset limpio
# df_limpio.to_csv('importaciones_limpio_para_ml.csv', index=False)


DataFrame Limpio y Preparado (Primeras filas):
    Año  mes NCM_8_digitos Pais_Origen_Cod  Peso_Neto_kg  FOB(USD)  \
0  2024    1      01012100             200         500.0   6000.00   
1  2024    1      01012100             212        2500.0  19500.00   
2  2024    1      01012100             417         300.0  33490.76   
3  2024    1      01012900             200         450.0   5000.00   
4  2024    1      01012900             423        1000.0   9800.00   

   Flete(USD)  Seguro(USD)  CIF(USD)  Precio_Unitario_USD_kg NCM_2_digitos  \
0     3500.00        47.50   9547.50               12.000000            01   
1    26000.01       455.01  45955.02                7.800000            01   
2     3256.80       367.48  37115.04              111.635867            01   
3     2500.00        75.00   7575.00               11.111111            01   
4     6513.60       168.27  16481.87                9.800000            01   

  NCM_4_digitos  
0          0101  
1          0101  
2       

# GUARDO EL ARCHIVO PARA PROXIMO PASO DE ANALISIS EXPLORATORIO DE DATOS (EDA)

In [26]:
# Guardo el DataFrame limpio en un nuevo archivo CSV
nombre_archivo_salida = 'impom24_listo_para_EDA.csv'

# index=False evita que Pandas guarde la columna de índice numérico por defecto
# decimal='.' es el estándar para CSVs si planeamos usarlo en otros softwares de ML
df_limpio.to_csv(nombre_archivo_salida, index=False, decimal='.')

print(f"✅ ¡Dataset limpio guardado con éxito como '{nombre_archivo_salida}'!")
print("Ya se puede empezar con el Análisis Exploratorio de Datos (EDA) en un nuevo notebook.")

✅ ¡Dataset limpio guardado con éxito como 'impom24_listo_para_EDA.csv'!
Ya se puede empezar con el Análisis Exploratorio de Datos (EDA) en un nuevo notebook.
